In [1]:
import pandas as pd

In [ ]:
excel_df = pd.read_excel('dataset.xlsx', engine='openpyxl')

In [ ]:
# create engine that will connect to our database with SQLAlchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data.db')

with engine.connect() as connection:
    sql_df = pd.read_sql_query(query, connection)

In [ ]:
# list of columns
df.columns

**Pandas Reference** <br>
DataFrames: > 1 column <br>
Series: only 1 column <br>

Object datatype: string or mixed values of strings and numbers

null vs na in pandas: https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas

axis 0: rows <br>
axis 1: columns

inplace=True changes the DataFrame itself instead of returning a new, modified DataFrame

**EDA**

Examine the top and bottom of the data <br>
Examine the data's dimensions <br>
Examine the datatypes and missing values <br>
Investigate statistical properties of the data <br>
Create plots of the data 

In [ ]:
# select by row number
df.iloc[0]
df.iloc[-1]
df.iloc[0,0]
df = df.iloc[:-5] # get rid of last 5 rows 

# select by index value
# NOTE: DataFrame indexes don't have to be unique – they can have repeated values
df.loc[100]

# select by column name
df[['column1', 'column2']]

In [ ]:
# If duplicate index values, we can change our index to be unique, sequential numbers
df.reset_index(inplace=True, drop=True)

In [ ]:
# check datatypes and missing values
df.info()

In [ ]:
# number of missing values
df.isna().sum()

In [ ]:
# statistics (for numerical features)
df.describe()

# For non-numeric columns, we can check distribution
df['column1'].mode()
df['column1'].value_counts()
df['column1'].value_counts()[:5] # check top 5

In [ ]:
# investigate how many unique items there are
df['column1'].unique().shape

In [ ]:
# correlations
# NOTE: If your data has non-linear relationships, then Pearson correlations are not the best method to use.
df.corr()

In [ ]:
import matplotlib.pyplot as plt
# plot histogram
# bins: number of bars
df['column1'].hist(bins=30)
plt.show()

# plot multiple histograms in separate subplots
df[['col1' , 'col2']].df.hist() 
# plot multiple histograms in a single plot
df.plot.hist()

In [ ]:
# scatterplot
df.plot.scatter(x='col1', y='col2')
plt.show()

In [ ]:
# bar plot of non-numeric data
df['col1'].value_counts().plot.bar()
plt.show()

**Cleaning Data**

- Removing irrelevant data
- Dealing with missing values (filling in or dropping them)
- Dealing with outliers
- Dealing with duplicate values
- Ensuring datatypes are correct
- Standardizing data formats (e.g. mismatched capitalization, converting units)

In [ ]:
# filtering data
df[df['col1'] > 100]

df[(df['col1'] > 100) & (df['col2'] > 200)]['col3'].value_counts()

Removing irrelevant data

In [ ]:
# drop columns
df.drop(['col1', 'col2'], axis=1, inplace=True)

# filter out genres that are not music
only_music = df[~df['Genre'].isin(['Drama', 'TV'])]

Dealing with missing values

examine the data with EDA and check any documentation on the data to see if missing values are represented in a special way

might be represented as NA, NaN, None or -999

Handling methods:
- Leave the missing values as-is
- Drop the data
- Fill with a specific value
- Replace with the mean, median, or mode
- Use machine learning to replace missing values (Imputer)

In [ ]:
# examine missing rows
# sample 5 missing data in col1
df[df['col1'].isna()].sample(5, random_state=42).head()

In [ ]:
# drop data
df.dropna(inplace=True)

In [ ]:
# fill in with specific value
df['col1'].fillna('Unknown', inplace=True)

In [ ]:
# replace with mode
# make sense if we have a dataset where most of the values are a particular value
df['col1'].fillna(df['col1'].mode(), inplace=True)

# fill with the mean (average)
# makes sense if we have a distribution of values that is somewhat Gaussian or a normal distribution

# fill with median if we have skewed distribution
# why? In the skewed distribution, the median is closer to the peak of the histogram

In [ ]:
# Use Machine Learning
# sklearn can only handle numeric data without missing values, so we can't pass it any strings
from sklearn.impute import KNNImputer
imputer = KNNImputer()
imputed = imputer.fit_transform(df[['col1', 'col2']])

# overwrite the target column with our new data
df['col1'] = imputed[:, 'col1']

Dealing with outliers

**Categorical** <br>
- remove these rows with filtering
- group all minority classes into a class we label as Other.

Dealing with categorical outliers mostly has little impact

**Numerical**
- interquartile range (IQR) 
- z-score methods

More methods: https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

In [ ]:
# IQR method
def remove_outlier(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    upper_boundary = q3 + 1.5*iqr
    lower_boundary = q1 - 1.5*iqr
    new_df = df.loc[(df[column] > lower_boundary) & (df[column] < upper_boundary)]
    return new_df

Dealing with duplicate values

In [ ]:
# check for duplicated values
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

Ensuring datatypes are correct
- object (for strings)
- int
- float

In [ ]:
df['col1'] = df['col1'].astype('int')

Standardizing data formats <br>
Male, male, M - all refer to the same thing

In [ ]:
# replace several data at once
# replace: non-matching values left alone
# map: non-matching values converted to NaN
genre_dict = {'metal': 'Metal', 'met': 'Metal'}
songs_df['Genre'].replace(genre_dict)

In [ ]:
df['col1'].apply(lambda x: x.lower())

Group by

In [ ]:
song_df.groupby('Genre').mean()['Seconds'].sort_values().head()

After preprocessing, save pandas dataframe as:
- csv
- HDF: can retrieve only parts of the data at a time (via the index)
- Feather: fast, compressed, and were designed for passing data between R and Python

In [ ]:
# write dataframe to disk
df.to_csv('file.csv', index=False)